In [1]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar
Finished download of spark-kernel-brunel-all-2.3.jar


In [2]:
//import libraries
import org.apache.spark.{SparkConf, SparkContext, SparkFiles}
import org.apache.spark.sql.{SQLContext, SparkSession, Row}
import org.apache.spark.SparkFiles

import org.apache.spark.ml.feature.{StringIndexer, IndexToString, VectorIndexer, VectorAssembler}
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.classification.{LogisticRegression, DecisionTreeClassifier}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.{Pipeline, PipelineStage}

# Loading transactions

In [3]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._ 

val toDouble = udf {x: Boolean => if(x) 1.0 else 0.0}

//Load data from DB2 for z/OS using JDBC driver
val orderDetailsDF = spark.
    read.format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").
    option("header", "true").
    option("inferSchema", "true").
    load("./assets/repellent-data-set.csv")
val orderDetailsData = orderDetailsDF.withColumn("label", toDouble(orderDetailsDF("REPELLENT"))).
                                      drop("CITY", "STATE", "COUNTRY", "TENT_NEGTWEETS", "REPELLENT", "TENT") 

orderDetailsData.show(5)

+------+---+--------------+------------+-------------------+-----+
|GENDER|AGE|MARITAL_STATUS|  PROFESSION|REPELLENT_NEGTWEETS|label|
+------+---+--------------+------------+-------------------+-----+
|     F| 39|   Unspecified| Hospitality|                  6|  0.0|
|     F| 37|       Married|      Retail|                  7|  0.0|
|     F| 24|        Single|       Other|                  7|  0.0|
|     F| 34|       Married|Professional|                  2|  1.0|
|     F| 22|       Married|       Other|                  2|  1.0|
+------+---+--------------+------------+-------------------+-----+
only showing top 5 rows



# Splitting into traning and testing data sets

In [4]:
val train = 0.85
val test = 0.15

//Split the data into training data set, testing data set, and validation data set

val splits = orderDetailsData.randomSplit(Array(train, test), seed = 17L)

val trainingDF = splits(0)
val testDF = splits(1)

println("Training data set")
trainingDF.show(5)

println("Validation data set")
testDF.show(5)

Training data set
+------+---+--------------+-----------+-------------------+-----+
|GENDER|AGE|MARITAL_STATUS| PROFESSION|REPELLENT_NEGTWEETS|label|
+------+---+--------------+-----------+-------------------+-----+
|     F| 19|        Single|Hospitality|                  6|  0.0|
|     F| 19|        Single|      Other|                  1|  1.0|
|     F| 19|        Single|      Other|                  7|  0.0|
|     F| 19|        Single|      Other|                  9|  0.0|
|     F| 19|        Single|    Student|                  2|  1.0|
+------+---+--------------+-----------+-------------------+-----+
only showing top 5 rows

Validation data set
+------+---+--------------+------------+-------------------+-----+
|GENDER|AGE|MARITAL_STATUS|  PROFESSION|REPELLENT_NEGTWEETS|label|
+------+---+--------------+------------+-------------------+-----+
|     F| 19|        Single|Professional|                  1|  1.0|
|     F| 20|       Married|       Other|                  3|  1.0|
|     F|

# Building Features

In [5]:
import org.apache.spark.ml.feature.{StringIndexer, IndexToString, VectorIndexer, VectorAssembler}

//Feature definition

val genderIndexer = new StringIndexer().setInputCol("GENDER").setOutputCol("gender_code")
val maritalStatusIndexer = new StringIndexer().setInputCol("MARITAL_STATUS").setOutputCol("marital_status_code")
val professionIndexer = new StringIndexer().setInputCol("PROFESSION").setOutputCol("profession_code")

val featuresAssembler = new VectorAssembler().setInputCols(Array("AGE",
                                                                 "gender_code",
                                                                 "marital_status_code",
                                                                 "profession_code",
                                                                 "REPELLENT_NEGTWEETS")).setOutputCol("features")

# Assembling a pipeline with logistic regression model

In [6]:
import org.apache.spark.ml.classification.{LogisticRegression, DecisionTreeClassifier}

//Using Logistic Regression
val lr = new LogisticRegression().setRegParam(0.01).setLabelCol("label").setFeaturesCol("features")


In [7]:
import org.apache.spark.ml.{Pipeline, PipelineStage}


//Cognitive Assistant for Data Scientists - predict model performance based on sampled data
val pipeline = new Pipeline().setStages(Array(genderIndexer, 
                                              maritalStatusIndexer, 
                                              professionIndexer,
                                              featuresAssembler,
                                              lr))
val model = pipeline.fit(trainingDF)


# Using RDD-based API for assessing performance

In [8]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint


val testDFWithPredictions = model.transform(testDF)
testDFWithPredictions.show(5)

// Compute raw scores on the test set
val predictionAndLabels = testDFWithPredictions.select("prediction", "label").
                                                rdd.map { row =>
  (row(0).asInstanceOf[Double], row(1).asInstanceOf[Double])
}

predictionAndLabels.take(5).foreach(println)


+------+---+--------------+------------+-------------------+-----+-----------+-------------------+---------------+--------------------+--------------------+--------------------+----------+
|GENDER|AGE|MARITAL_STATUS|  PROFESSION|REPELLENT_NEGTWEETS|label|gender_code|marital_status_code|profession_code|            features|       rawPrediction|         probability|prediction|
+------+---+--------------+------------+-------------------+-----+-----------+-------------------+---------------+--------------------+--------------------+--------------------+----------+
|     F| 19|        Single|Professional|                  1|  1.0|        1.0|                1.0|            4.0|[19.0,1.0,1.0,4.0...|[-2.6929914355695...|[0.06338818430343...|       1.0|
|     F| 20|       Married|       Other|                  3|  1.0|        1.0|                0.0|            0.0|[20.0,1.0,0.0,0.0...|[-0.9325964119808...|[0.28239825582197...|       1.0|
|     F| 20|       Married|      Retail|               

## Metrics (RDD-based API)

In [9]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics

// Instantiate metrics object
val metrics = new MulticlassMetrics(predictionAndLabels)

// Confusion matrix
println("Confusion matrix:")
println(metrics.confusionMatrix)

Confusion matrix:
269.0  4.0    
13.0   105.0  


In [10]:
val accuracy = metrics.accuracy
println("Statistics")
println(s"Accuracy = $accuracy")

// Precision by label
val labels = metrics.labels
labels.foreach { label =>
  println(s"Precision($label) = " + metrics.precision(label))
}

// Recall by label
labels.foreach { label =>
  println(s"Recall($label) = " + metrics.recall(label))
}

// False positive rate by label
labels.foreach { label =>
  println(s"FPR($label) = " + metrics.falsePositiveRate(label))
}

// F-measure by label
labels.foreach { label =>
  println(s"F1-Score($label) = " + metrics.fMeasure(label))
}

// Weighted statistics
println(s"Weighted precision: ${metrics.weightedPrecision}")
println(s"Weighted recall: ${metrics.weightedRecall}")
println(s"Weighted F1 score: ${metrics.weightedFMeasure}")
println(s"Weighted false positive rate: ${metrics.weightedFalsePositiveRate}")

Statistics
Accuracy = 0.9565217391304348
Precision(0.0) = 0.9539007092198581
Precision(1.0) = 0.963302752293578
Recall(0.0) = 0.9853479853479854
Recall(1.0) = 0.8898305084745762
FPR(0.0) = 0.11016949152542373
FPR(1.0) = 0.014652014652014652
F1-Score(0.0) = 0.9693693693693693
F1-Score(1.0) = 0.9251101321585902
Weighted precision: 0.956738154444152
Weighted recall: 0.9565217391304348
Weighted F1 score: 0.9560123617200804
Weighted false positive rate: 0.08134324530787315


# Using DataFrame-based API for assessing performance

In [11]:
import org.apache.spark.ml.classification.{BinaryLogisticRegressionSummary, LogisticRegressionModel}

// Extract the summary from the LogisticRegressionModel instance 
val lrModel = model.stages(4).asInstanceOf[LogisticRegressionModel]
val trainingSummary = lrModel.summary

// Metrics for binary classification
val binarySummary = trainingSummary.asInstanceOf[BinaryLogisticRegressionSummary]

// The receiver-operating characteristic curve and area under the ROC curve
val roc = binarySummary.roc
println(s"areaUnderROC: ${binarySummary.areaUnderROC}")

// Example of optimizing logistic regression model to maximize F-measure
val fMeasure = binarySummary.fMeasureByThreshold
fMeasure.show(5)
val maxFMeasure = fMeasure.select(max("F-Measure")).head().getDouble(0)
val bestThreshold = fMeasure.where(fMeasure("F-Measure") === maxFMeasure).
  select("threshold").head().getDouble(0)
println(s"best threshold:${bestThreshold}")

val updatedModel = lrModel.setThreshold(bestThreshold)     

areaUnderROC: 0.9908196261965346
+------------------+-------------------+
|         threshold|          F-Measure|
+------------------+-------------------+
|0.9799516477338245|0.05499276410998553|
|0.9787466710287548|0.13592233009708737|
|0.9776017293505477| 0.1959731543624161|
|0.9765901367896532|0.24313725490196078|
|0.9663111958732358|0.29441624365482233|
+------------------+-------------------+
only showing top 5 rows

best threshold:0.48423523071330815


## ROC curve based on training data

In [12]:
%%brunel data('roc') x(FPR) y(TPR) line tooltip(#all) axes(x:'False Positive Rate':grid, y:'True Positive Rate':grid) title('ROC') 

## ROC curve based on test data

In [13]:
val testData = testDFWithPredictions.drop("prediction", "rawPrediction", "probability")
val trainingSummary = lrModel.evaluate(testData)
val binarySummary = trainingSummary.asInstanceOf[BinaryLogisticRegressionSummary]

// The receiver-operating characteristic curve and area under the ROC curve
val rocOnTestData = binarySummary.roc
println(s"areaUnderROC: ${binarySummary.areaUnderROC}")

// The threshold to maximize F-Measure
val fMeasure = binarySummary.fMeasureByThreshold
val maxFMeasure = fMeasure.select(max("F-Measure")).head().getDouble(0)
val bestThreshold = fMeasure.where(fMeasure("F-Measure") === maxFMeasure).
  select("threshold").head().getDouble(0)
println(s"best threashold:${bestThreshold}")

areaUnderROC: 0.9912460420934996
best threashold:0.47762508181555036


In [14]:
%%brunel data('rocOnTestData') x(FPR) y(TPR) line tooltip(#all) axes(x:'False Positive Rate':grid, y:'True Positive Rate':grid) title('ROC') 

Prepared by Aleksandr Petrov, Analytics Group Research Team, CPO